In [1]:
import os

# THIRD PARTY PACKAGES
import dotenv    # To load environment variables from .env file (https://github.com/theskumar/python-dotenv)
import gspread   # To access Google Sheets (https://gspread.readthedocs.io/en/latest/)
import html2text # To convert HTML to text (https://github.com/Alir3z4/html2text/)
import jinja2    # To render HTML templates (https://jinja.palletsprojects.com/en/2.11.x/)
import sendgrid  # To send emails (https://sendgrid.com/docs/for-developers/sending-email/v3-python-code-example/)

In [2]:
# load any .env files around
dotenv.load_dotenv()

# api key (must be configured in Codespaces encrypted secrets)
SENDGRID_API_KEY = os.environ.get("SENDGRID_API_KEY")

# spreadsheet and worksheet to use
SSHEET_KEY = "11aR5WHloNn4N3Ns960reTCbxB_QUHeOU7RSqJzUyRbY"
SSHEET_WORKSHEET_ID = 1

# email information
EMAIL_FROM_SENDER = "Jérémie Lumbroso <lumbroso@princeton.edu>"
EMAIL_DEFAULT_SUBJECT = "Please {first_name}: Could you fill this form out by Wed?"

In [3]:
# sanity check
if SENDGRID_API_KEY is None:
    raise ValueError(
        "SENDGRID_API_KEY is not properly configured; "
        "if you are running this in a Codespace, please configure it in the encrypted secrets."
    )

In [4]:
# creating client using our API key (requires either local file,
# or GitHub secret configuration)

gc = gspread.service_account(filename="service_account.json")

# retrieving spreadsheet

ssheet = gc.open_by_key(SSHEET_KEY)
if ssheet is None:
    raise Exception("Spreadsheet not found")

# printing spreadsheet info

print("Sheet title:", ssheet.title)
print("Worksheets:")
for i, wks in enumerate(ssheet.worksheets()):
    print(i, wks.title)

# retrieving worksheet

ws = ssheet.get_worksheet(SSHEET_WORKSHEET_ID)

Sheet title: Main List of platforms from Gleidson/Media Cloud/iOS+Android Apps
Worksheets:
0 Main
1 email_list_final
2 email_list
3 missing_ones_Owen
4 dictionary completed_restaurant_extraction
5 temp
6 Final
7 descriptive_statistics_scraping


In [5]:
# get all rows
rows = ws.get_all_values()

# separate header
header = rows[0]
rows = rows[1:]

# zip records
records = [dict(zip(header, row)) for row in rows]

In [6]:
# See here for a primer on Jinja2:
# https://realpython.com/primer-on-jinja-templating/#render-your-first-jinja-template

environment = jinja2.Environment(loader=jinja2.FileSystemLoader("templates/"))

def render_template(template_file_name, **variables):
    template = environment.get_template(template_file_name)
    content = template.render(
        **variables
    )
    return content

In [7]:
# Records are of his format:
#
#  {'platform name': '501 Delivery',
#   'contact person': 'Marilyn Choke Smith',
#   'contact person email': '501deliveryservices@gmail.com',
#   'RMDA?': 'yes'},
#  {'platform name': '50 Grub 4 U',
#   'contact person': '',
#   'contact person email': '50grub4u@gmail.com',
#   'RMDA?': 'yes'},
#
# - some may not have a contact person
# - some may not have an email
# - some may have 'yes' or 'no in "RDMA?"

def contact_record(
    record,
    subject=None,
):

    # extract fields
    contact_person = record.get("contact person", "").strip()
    contact_email = record.get("contact person email", "").strip()

    # exit if no email
    if contact_email == "":
        return

    # determine first name
    first_name = ""
    if contact_person != "":
        first_name = record.get("contact person").split()[0]
    
    # to object
    to_obj = contact_email
    if contact_person != "":
        to_obj = (contact_email, contact_person)

    # pick the template as a function of the "RDMA?" field
    is_rmda = record.get("RMDA?", "").strip().lower() == "yes"
    template_file_name = "non_rmda_email.html"
    if is_rmda:
        template_file_name = "rmda_email.html"

    # render template and compute text version
    html_body = render_template(
        template_file_name=template_file_name,
        first_name=first_name,
    )
    txt_body = html2text.HTML2Text().handle(html_body)

    # render subject
    email_subject = subject or EMAIL_DEFAULT_SUBJECT.format(first_name=first_name)

    # create SendGrid message
    message = sendgrid.helpers.mail.Mail(
        from_email=EMAIL_FROM_SENDER,
        to_emails=[to_obj],
        subject=email_subject,
        html_content=html_body,
        plain_text_content=txt_body,
    )

    #message.cc = "Some Person <some@email.com>"
    #message.reply_to = "Some Person <some@email.com>"

    # remove tracking data to make email seem less spammy
    tracking_settings = sendgrid.helpers.mail.TrackingSettings()
    tracking_settings.click_tracking = sendgrid.helpers.mail.ClickTracking(
        enable=False,
        enable_text=False
    )
    tracking_settings.open_tracking = sendgrid.helpers.mail.OpenTracking(
        enable=False
    )
    message.tracking_settings = tracking_settings

    # Add unsubsubscribe header to seem less spammy
    message.add_header(
        sendgrid.helpers.mail.Header("List-Unsubscribe", EMAIL_FROM_SENDER)
    )
    
    # Create the SendGrid client and send the email
    sg = sendgrid.SendGridAPIClient(SENDGRID_API_KEY)
    response = sg.send(message)

    # Print log
    #print(message)
    #print(response.status_code, response.body, response.headers)

    return response.status_code == 202

In [8]:
# just an example
test_record = {
   'platform name': '501 Delivery',
   'contact person': 'John Smith',
   'contact person email': 'lumbroso@cs.princeton.edu',
   'RMDA?': 'yes'
}
contact_record(test_record)

TemplateNotFound: rmda_email.html